In [1]:
# Optional dependency check (avoid installing into /home on clusters)
import importlib.util
print("accelerate available:", importlib.util.find_spec("accelerate") is not None)


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os

# Fix cluster home quota: redirect caches (edit path if needed)
os.environ.setdefault("ROLE_REP_CACHE_DIR", f"/projects/JeFeSpace/KLM/cache/{os.environ.get('USER','user')}/role-rep")
os.environ.setdefault("HF_HOME", os.path.join(os.environ["ROLE_REP_CACHE_DIR"], "hf"))
os.environ.setdefault("HUGGINGFACE_HUB_CACHE", os.path.join(os.environ["HF_HOME"], "hub"))
os.environ.setdefault("TRANSFORMERS_CACHE", os.path.join(os.environ["HF_HOME"], "transformers"))
os.environ.setdefault("TORCH_HOME", os.path.join(os.environ["ROLE_REP_CACHE_DIR"], "torch"))
os.environ.setdefault("XDG_CACHE_HOME", os.path.join(os.environ["ROLE_REP_CACHE_DIR"], "xdg"))

# Avoid widget/progress-bar issues + HF xet
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["DISABLE_TQDM"] = "1"
os.environ["HF_HUB_DISABLE_XET"] = "1"

import getpass
if not (os.environ.get("HF_TOKEN") or os.environ.get("HUGGINGFACE_HUB_TOKEN")):
    os.environ["HF_TOKEN"] = getpass.getpass("HF token: ")
    os.environ["HUGGINGFACE_HUB_TOKEN"] = os.environ["HF_TOKEN"]

HF token:  ········


# Speaker Role Representation: Geometric Structure Analysis

**Goal:** Compare how two models (Qwen vs Llama) internally represent speaker identity in a long two‑person transcript.

We treat **speaker identity (Alice vs Bob)** as the concept and probe activations across layers and token positions.
Deliverables: PCA/t‑SNE, separability heatmaps, and concept directions.


**Question:** Do models maintain distinct internal representations for different speakers over a long conversation?

**Setup:** We build a long two‑person transcript (Alice vs Bob). Each token is labeled by the most recent speaker tag.
We extract hidden states across layers and token positions (speaker tag, subject tokens, verbs, final token).

**Analyses:**
- **Geometry:** PCA/t‑SNE plots show whether Alice and Bob tokens cluster.
- **Separability:** Linear classifiers measure how well a simple boundary separates speakers.
- **Concept direction:** Mean‑difference vectors test if a consistent "speaker axis" exists.

**Comparison:** Run the same pipeline on **Qwen** and **Llama** to compare where and how the concept emerges.

**Takeaway:** The layer and position with the cleanest separation indicate where speaker identity is most strongly encoded.


In [3]:
# %%capture
# Core
import json
import math
import os
import sys
from dataclasses import dataclass
from typing import Dict, List

# --- Cache + auth (prevents /home quota issues on clusters) ---
# You can override explicitly with ROLE_REP_CACHE_DIR.
ROLE_REP_CACHE_DIR = os.environ.get("ROLE_REP_CACHE_DIR")
if ROLE_REP_CACHE_DIR is None:
    user = os.environ.get("USER", "user")
    if os.path.isdir("/projects/JeFeSpace/KLM"):
        ROLE_REP_CACHE_DIR = f"/projects/JeFeSpace/KLM/cache/{user}/role-rep"
    elif os.path.isdir("/scratch"):
        ROLE_REP_CACHE_DIR = f"/scratch/{user}/role-rep-cache"

if ROLE_REP_CACHE_DIR:
    os.makedirs(ROLE_REP_CACHE_DIR, exist_ok=True)
    os.environ.setdefault("HF_HOME", os.path.join(ROLE_REP_CACHE_DIR, "hf"))
    os.environ.setdefault("HUGGINGFACE_HUB_CACHE", os.path.join(os.environ["HF_HOME"], "hub"))
    os.environ.setdefault("TRANSFORMERS_CACHE", os.path.join(os.environ["HF_HOME"], "transformers"))
    os.environ.setdefault("TORCH_HOME", os.path.join(ROLE_REP_CACHE_DIR, "torch"))
    os.environ.setdefault("XDG_CACHE_HOME", os.path.join(ROLE_REP_CACHE_DIR, "xdg"))

# Reduce notebook widget/progress issues
os.environ.setdefault("HF_HUB_DISABLE_PROGRESS_BARS", "1")
# Avoid HF "xet" download path (often problematic on clusters)
os.environ.setdefault("HF_HUB_DISABLE_XET", "1")

# If you installed packages into project space (e.g., accelerate), add that site-packages BEFORE importing transformers.
ROLE_REP_SITE_PACKAGES = os.environ.get(
    "ROLE_REP_SITE_PACKAGES",
    "/projects/JeFeSpace/KLM/pip_local/lib/python3.12/site-packages",
)
if os.path.isdir(ROLE_REP_SITE_PACKAGES) and ROLE_REP_SITE_PACKAGES not in sys.path:
    sys.path.insert(0, ROLE_REP_SITE_PACKAGES)


# If you exported HF_TOKEN/HUGGINGFACE_HUB_TOKEN in your shell, this will be picked up automatically.
HF_TOKEN = os.environ.get("HF_TOKEN") or os.environ.get("HUGGINGFACE_HUB_TOKEN")

# ML / viz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Hugging Face
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Repro
np.random.seed(7)
torch.manual_seed(7)


/shared/EL9/explorer/anaconda3/2024.06/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
# Transcript: long two-person dialogue (Alice/Bob)
transcript_turns = [
    "Alice: Hey Bob, before we jump in, I wanted to revisit the design proposal.",
    "Bob: Sure, I skimmed it last night; the latency targets look ambitious.",
    "Alice: The client asked for sub-200ms p95; I think batching can get us there.",
    "Bob: Batching helps, but the cache invalidation could become tricky.",
    "Alice: We can constrain invalidation to product-level keys instead of per-user.",
    "Bob: That might reduce precision, though; would that impact personalization?",
    "Alice: Some, but we can re-rank on the client side for the top 10 results.",
    "Bob: Okay, so you are proposing a hybrid: coarse cache, fine rerank.",
    "Alice: Exactly. And we should log enough to measure drift each week.",
    "Bob: Logging is fine, but data retention policy caps at 30 days.",
    "Alice: Right, I can summarize weekly aggregates and delete raw events.",
    "Bob: Great. Also, the new API endpoint needs a version bump.",
    "Alice: v3 seems reasonable; we can keep v2 for a deprecation window.",
    "Bob: Then we need a migration guide; I can draft it.",
    "Alice: Thanks. Another point: the search index rebuild takes 6 hours.",
    "Bob: Maybe we can parallelize by shard and compress the postings lists.",
    "Alice: If we compress too much, we might slow decoding at query time.",
    "Bob: True; we could trade storage for CPU if latency budget allows.",
    "Alice: We'll benchmark both. Also, what about adding synonyms?",
    "Bob: Synonyms help recall, but they increase false positives.",
    "Alice: We'll tune the threshold and evaluate per-category.",
    "Bob: Sounds good. On another note, QA reported flaky tests.",
    "Alice: I saw that; I think the mock clock isn't resetting in CI.",
    "Bob: I can isolate those tests and add a fixture.",
    "Alice: Appreciate it. Lastly, are we aligned on the rollout plan?",
    "Bob: Staged rollout: internal, then 5% external, then 50%.",
    "Alice: And we monitor error rates and rollback if p95 spikes.",
    "Bob: Yes. I'll write the runbook.",
    "Alice: Great, I'll update the proposal and send it today.",
    "Bob: Perfect; I will review as soon as it lands.",
    "Alice: Thanks, Bob.",
    "Bob: Thanks, Alice."
]

transcript = "\n".join(transcript_turns)
transcript

"Alice: Hey Bob, before we jump in, I wanted to revisit the design proposal.\nBob: Sure, I skimmed it last night; the latency targets look ambitious.\nAlice: The client asked for sub-200ms p95; I think batching can get us there.\nBob: Batching helps, but the cache invalidation could become tricky.\nAlice: We can constrain invalidation to product-level keys instead of per-user.\nBob: That might reduce precision, though; would that impact personalization?\nAlice: Some, but we can re-rank on the client side for the top 10 results.\nBob: Okay, so you are proposing a hybrid: coarse cache, fine rerank.\nAlice: Exactly. And we should log enough to measure drift each week.\nBob: Logging is fine, but data retention policy caps at 30 days.\nAlice: Right, I can summarize weekly aggregates and delete raw events.\nBob: Great. Also, the new API endpoint needs a version bump.\nAlice: v3 seems reasonable; we can keep v2 for a deprecation window.\nBob: Then we need a migration guide; I can draft it.\nA

In [5]:
# Model config: Qwen 7B + Qwen 2.5 + Llama 3.1 8B
# These are HF names; swap to local paths if you have them.

MODEL_NAMES = {
    "qwen_7b": os.environ.get("QWEN_7B_MODEL", "Qwen/Qwen2-7B-Instruct"),
    "qwen_2_5": os.environ.get("QWEN_25_MODEL", "Qwen/Qwen2.5-7B-Instruct"),
    # NOTE: official HF repo name often includes "Meta-" prefix
    "llama_3_1_8b": os.environ.get("LLAMA_31_8B_MODEL", "meta-llama/Meta-Llama-3.1-8B-Instruct"),
}

# Some clusters ship an older transformers that can't parse Llama-3.1 rope_scaling.
# This shim overrides rope_scaling into the older (type,factor) format so the model can load.
MODEL_LOAD_KWARGS = {
    "llama_3_1_8b": {
        "rope_scaling": {"type": "linear", "factor": 8.0},
    }
}

# Layers to probe: 0, 25%, 50%, 75%, 100%
LAYER_FRACTIONS = [0.0, 0.25, 0.5, 0.75, 1.0]

# Token positions to probe
POSITION_SPECS = {
    "final": {"type": "final"},
    "speaker_tag": {"type": "string", "values": ["Alice", "Bob"]},
    "verb": {"type": "string", "values": ["am", "will", "can", "should", "think", "like", "want"]},
}


In [6]:
@dataclass
class ModelBundle:
    name: str
    tokenizer: AutoTokenizer
    model: AutoModelForCausalLM


def _best_dtype():
    # Use bf16 on A100/H100 when available; otherwise fall back.
    if torch.cuda.is_available():
        try:
            major, _minor = torch.cuda.get_device_capability(0)
            if major >= 8:  # Ampere+
                return torch.bfloat16
        except Exception:
            pass
        return torch.float16
    return torch.float32


def _load_config_with_rope_compat(model_id: str, cache_dir: str | None):
    """Load config.json as dict, patch rope_scaling for older transformers, then build config."""
    import json
    from transformers import AutoConfig
    from transformers.utils import cached_file

    cfg_path = cached_file(model_id, "config.json", cache_dir=cache_dir, token=HF_TOKEN)
    with open(cfg_path, "r") as f:
        cfg = json.load(f)

    rs = cfg.get("rope_scaling")
    # Llama-3.1 uses a richer rope_scaling schema (rope_type/low_freq_factor/etc.)
    # Older transformers expects exactly {type, factor}.
    if isinstance(rs, dict) and ("type" not in rs) and ("rope_type" in rs):
        cfg["rope_scaling"] = {"type": "linear", "factor": float(rs.get("factor", 8.0))}

    return AutoConfig.from_dict(cfg)


def load_model(name: str, model_kwargs: Dict | None = None) -> ModelBundle:
    import importlib.util

    cache_dir = os.environ.get("TRANSFORMERS_CACHE") or os.environ.get("HF_HOME")
    dtype = _best_dtype()
    model_kwargs = model_kwargs or {}

    tokenizer = AutoTokenizer.from_pretrained(
        name,
        cache_dir=cache_dir,
        token=HF_TOKEN,
    )

    have_accelerate = importlib.util.find_spec("accelerate") is not None
    device_map = "auto" if (torch.cuda.is_available() and have_accelerate) else None
    low_cpu_mem_usage = True if have_accelerate else False

    try:
        model = AutoModelForCausalLM.from_pretrained(
            name,
            cache_dir=cache_dir,
            device_map=device_map,
            torch_dtype=dtype,
            low_cpu_mem_usage=low_cpu_mem_usage,
            token=HF_TOKEN,
            **model_kwargs,
        )
    except ValueError as e:
        # Handle Llama-3.1 rope_scaling schema mismatch on older transformers
        if "rope_scaling" in str(e) and "type" in str(e) and "factor" in str(e):
            cfg = _load_config_with_rope_compat(name, cache_dir)
            model = AutoModelForCausalLM.from_pretrained(
                name,
                cache_dir=cache_dir,
                config=cfg,
                device_map=device_map,
                torch_dtype=dtype,
                low_cpu_mem_usage=low_cpu_mem_usage,
                token=HF_TOKEN,
            )
        else:
            raise

    # If accelerate isn't available but CUDA is, move entire model to GPU.
    if torch.cuda.is_available() and device_map is None:
        model = model.to("cuda")

    # Ensure hidden states are returned
    model.config.output_hidden_states = True
    model.eval()

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return ModelBundle(name=name, tokenizer=tokenizer, model=model)


def select_layers(num_layers: int, fractions: List[float]) -> List[int]:
    indices = []
    for f in fractions:
        idx = int(round(f * num_layers))
        idx = max(0, min(num_layers, idx))
        indices.append(idx)
    return sorted(list(dict.fromkeys(indices)))


def find_positions(tokens: List[str], spec: Dict) -> List[int]:
    if spec["type"] == "final":
        return [len(tokens) - 1]
    if spec["type"] == "string":
        values = set(spec["values"])
        hits = [i for i, t in enumerate(tokens) if t.strip() in values]
        if hits:
            return hits
        return [i for i, t in enumerate(tokens) if t.strip().lstrip("Ġ").lower() in {v.lower() for v in values}]
    return []


def label_speakers(tokens: List[str], raw_text: str) -> np.ndarray:
    labels = []
    last_speaker = None
    cursor = 0

    for tok in tokens:
        window = raw_text[cursor: cursor + 200]
        if "Alice:" in window and ("Bob:" not in window or window.index("Alice:") < window.index("Bob:")):
            last_speaker = "Alice"
        if "Bob:" in window and ("Alice:" not in window or window.index("Bob:") < window.index("Alice:")):
            last_speaker = "Bob"

        labels.append(last_speaker if last_speaker else "Unknown")

        tok_clean = tok.replace("Ġ", " ")
        if tok_clean in raw_text[cursor:]:
            cursor = raw_text.index(tok_clean, cursor) + len(tok_clean)
        else:
            cursor = min(cursor + max(1, len(tok_clean)), len(raw_text))

    return np.array(labels)


In [7]:
def run_analysis_for_model(bundle: ModelBundle) -> Dict:
    tokenizer = bundle.tokenizer
    model = bundle.model

    encoded = tokenizer(transcript, return_tensors="pt")
    input_ids = encoded["input_ids"]

    # Try moving inputs to the model device when possible
    try:
        encoded = encoded.to(next(model.parameters()).device)
    except Exception:
        pass

    with torch.no_grad():
        outputs = model(**encoded, output_hidden_states=True, return_dict=True)

    hidden_states = outputs.hidden_states
    if hidden_states is None:
        raise RuntimeError("Model did not return hidden states; ensure output_hidden_states=True")

    num_layers = len(hidden_states) - 1
    layer_indices = select_layers(num_layers, LAYER_FRACTIONS)

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    positions = {name: find_positions(tokens, spec) for name, spec in POSITION_SPECS.items()}

    speaker_labels = label_speakers(tokens, transcript)

    data = {}
    for layer in layer_indices:
        hs = hidden_states[layer][0].detach().cpu().numpy()  # [seq_len, hidden_dim]
        for pos_name, pos_indices in positions.items():
            pos_indices = [p for p in pos_indices if p < hs.shape[0]]
            if not pos_indices:
                continue
            X = hs[pos_indices]
            y = speaker_labels[pos_indices]
            mask = np.isin(y, ["Alice", "Bob"])
            X = X[mask]
            y = y[mask]
            if len(y) < 4:
                continue
            data[(layer, pos_name)] = (X, y)

    return {
        "model_name": bundle.name,
        "tokens": tokens,
        "layers": layer_indices,
        "positions": positions,
        "speaker_labels": speaker_labels,
        "data": data,
    }


In [ ]:
# Run analysis model-by-model (avoids loading all 7B/8B models at once)

results = {}
for key, name in MODEL_NAMES.items():
    print(f"Loading {key}: {name}")
    bundle = load_model(name, model_kwargs=MODEL_LOAD_KWARGS.get(key))
    try:
        results[key] = run_analysis_for_model(bundle)
    finally:
        # Free GPU/CPU memory before next model
        try:
            del bundle.model
        except Exception:
            pass
        del bundle
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

list(results.keys())

Loading qwen_7b: Qwen/Qwen2-7B-Instruct


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Loading qwen_2_5: Qwen/Qwen2.5-7B-Instruct


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def plot_pca_tsne(model_key: str, res: Dict):
    plots = []
    for (layer, pos_name), (X, y) in res["data"].items():
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X)

        tsne = TSNE(n_components=2, perplexity=min(10, max(2, len(X) // 3)), init="random", random_state=7)
        X_tsne = tsne.fit_transform(X)

        plots.append(((layer, pos_name), X_pca, X_tsne, y))

    if not plots:
        print(f"No plots for {model_key}")
        return

    cols = 2
    rows = math.ceil(len(plots) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(10, 4 * rows))
    if rows == 1:
        axes = np.array([axes])

    for ax, ((layer, pos_name), X_pca, _, y) in zip(axes.flatten(), plots):
        colors = np.where(y == "Alice", "tab:blue", "tab:orange")
        ax.scatter(X_pca[:, 0], X_pca[:, 1], c=colors, alpha=0.7, s=30)
        ax.set_title(f"{model_key} PCA: layer {layer}, pos {pos_name}")
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")

    plt.tight_layout()
    plt.show()

    # t-SNE plots separately (can be slow)
    cols = 2
    rows = math.ceil(len(plots) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(10, 4 * rows))
    if rows == 1:
        axes = np.array([axes])

    for ax, ((layer, pos_name), _, X_tsne, y) in zip(axes.flatten(), plots):
        colors = np.where(y == "Alice", "tab:blue", "tab:orange")
        ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=colors, alpha=0.7, s=30)
        ax.set_title(f"{model_key} t-SNE: layer {layer}, pos {pos_name}")
        ax.set_xlabel("Dim1")
        ax.set_ylabel("Dim2")

    plt.tight_layout()
    plt.show()


for key, res in results.items():
    plot_pca_tsne(key, res)


In [ ]:
def compute_separability(res: Dict) -> pd.DataFrame:
    rows = []
    for (layer, pos_name), (X, y) in res["data"].items():
        y_bin = (y == "Alice").astype(int)
        n_splits = min(5, len(y_bin))
        if n_splits < 2:
            continue
        cv = StratifiedKFold(n_splits=n_splits)
        accs = []
        for train_idx, test_idx in cv.split(X, y_bin):
            clf = LogisticRegression(max_iter=1000)
            clf.fit(X[train_idx], y_bin[train_idx])
            preds = clf.predict(X[test_idx])
            accs.append(accuracy_score(y_bin[test_idx], preds))
        rows.append({"layer": layer, "position": pos_name, "accuracy": float(np.mean(accs))})
    return pd.DataFrame(rows)


def plot_heatmap(model_key: str, df: pd.DataFrame):
    if df.empty:
        print(f"No separability data for {model_key}")
        return
    pivot = df.pivot(index="layer", columns="position", values="accuracy")
    fig, ax = plt.subplots(figsize=(6, 4))
    im = ax.imshow(pivot.values, cmap="viridis", vmin=0.5, vmax=1.0)
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels(pivot.columns)
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels(pivot.index)
    ax.set_title(f"{model_key} separability (Alice vs Bob)")
    ax.set_xlabel("Token position")
    ax.set_ylabel("Layer")
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("Accuracy")
    plt.tight_layout()
    plt.show()


separability = {key: compute_separability(res) for key, res in results.items()}
for key, df in separability.items():
    display(df)
    plot_heatmap(key, df)


In [ ]:
def mean_difference_direction(X: np.ndarray, y: np.ndarray) -> np.ndarray:
    alice_mean = X[y == "Alice"].mean(axis=0)
    bob_mean = X[y == "Bob"].mean(axis=0)
    return alice_mean - bob_mean


def concept_direction_eval(res: Dict) -> pd.DataFrame:
    rows = []
    for (layer, pos_name), (X, y) in res["data"].items():
        direction = mean_difference_direction(X, y)
        proj = X @ direction
        threshold = np.median(proj)
        preds = (proj > threshold).astype(int)
        acc = accuracy_score((y == "Alice").astype(int), preds)
        rows.append({"layer": layer, "position": pos_name, "accuracy": float(acc)})
    return pd.DataFrame(rows)


def plot_concept_pca(model_key: str, res: Dict):
    # Visualize direction in PCA space for one example (highest separability layer/pos)
    df = compute_separability(res)
    if df.empty:
        return
    best = df.sort_values("accuracy", ascending=False).iloc[0]
    layer = int(best["layer"])
    pos_name = best["position"]
    X, y = res["data"][(layer, pos_name)]

    direction = mean_difference_direction(X, y)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    dir_pca = pca.transform(direction.reshape(1, -1))[0]

    colors = np.where(y == "Alice", "tab:blue", "tab:orange")
    plt.figure(figsize=(5, 4))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=colors, alpha=0.7, s=30)
    plt.arrow(0, 0, dir_pca[0], dir_pca[1], color="black", width=0.01)
    plt.title(f"{model_key} concept direction (layer {layer}, {pos_name})")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.tight_layout()
    plt.show()


concept_results = {key: concept_direction_eval(res) for key, res in results.items()}
for key, df in concept_results.items():
    display(df)
    plot_concept_pca(key, results[key])


## Write-up Template (fill after running)

**Key observations**
- Which layer shows the clearest Alice/Bob separation?
- Which position (speaker tags vs verbs vs final token) carries the cleanest signal?
- Do Qwen and Llama differ in where the signal peaks?

**Interpretation**
- Does the representation look linearly separable (high accuracy), or more diffuse?
- Is speaker identity localized to specific positions or distributed?
- Does speaker identity emerge late (higher layers) or early?
